In [2]:
import gc
import os
import h5py
import pickle
from partial import *

In [12]:
L = 31
bond = 150
p = partial(L = L, bond = bond)

In [13]:
keep = [0] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:01<00:00, 159.59it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:00<00:00, 491.56it/s]


Partial density matrices computed in 4.21 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 867996.06it/s]


Gram matrix for training set computed in 0.06 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 2256.28it/s]

Gram matrix for testing set computed in 0.18 seconds.


In [14]:
keep = [0,1] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 846.14it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 312.86it/s]


Partial density matrices computed in 3.24 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 573053.81it/s]


Gram matrix for training set computed in 0.09 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 2198.30it/s]

Gram matrix for testing set computed in 0.18 seconds.


In [15]:
keep = [0,1,2] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 877.57it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 277.82it/s]


Partial density matrices computed in 3.41 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 626312.96it/s]


Gram matrix for training set computed in 0.08 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1993.12it/s]

Gram matrix for testing set computed in 0.20 seconds.


In [16]:
keep = [0,1,2,3] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 824.79it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:00<00:00, 429.84it/s]


Partial density matrices computed in 3.40 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 632274.31it/s]


Gram matrix for training set computed in 0.08 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1682.83it/s]

Gram matrix for testing set computed in 0.24 seconds.


In [17]:
keep = [0,1,2,3,4] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 798.62it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 268.26it/s]


Partial density matrices computed in 3.51 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 287014.24it/s]


Gram matrix for training set computed in 0.17 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 702.01it/s]

Gram matrix for testing set computed in 0.57 seconds.


In [18]:
keep = [0,1,2,3,4,5] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 716.94it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 286.56it/s]


Partial density matrices computed in 3.77 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:01<00:00, 47984.30it/s]


Gram matrix for training set computed in 1.01 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:03<00:00, 133.10it/s]

Gram matrix for testing set computed in 3.01 seconds.


In [19]:
keep = [20] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 750.87it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 217.33it/s]


Partial density matrices computed in 4.48 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 453822.45it/s]


Gram matrix for training set computed in 0.11 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1386.87it/s]

Gram matrix for testing set computed in 0.29 seconds.


In [20]:
keep = [20,21] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 696.60it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 202.08it/s]


Partial density matrices computed in 4.61 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 444401.35it/s]


Gram matrix for training set computed in 0.11 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1390.00it/s]

Gram matrix for testing set computed in 0.29 seconds.


In [21]:
keep = [20,21,22] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 694.51it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 200.28it/s]


Partial density matrices computed in 4.54 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 440986.82it/s]


Gram matrix for training set computed in 0.11 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1492.82it/s]

Gram matrix for testing set computed in 0.27 seconds.


In [22]:
keep = [20,21,22,23] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 652.94it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 201.26it/s]


Partial density matrices computed in 4.77 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 395997.03it/s]


Gram matrix for training set computed in 0.13 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1159.78it/s]

Gram matrix for testing set computed in 0.35 seconds.


In [23]:
keep = [20,21,22,23,24] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 697.78it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:02<00:00, 196.32it/s]


Partial density matrices computed in 4.60 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 207757.60it/s]


Gram matrix for training set computed in 0.24 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 609.10it/s]

Gram matrix for testing set computed in 0.66 seconds.


In [25]:
keep = [20,21,22,23,24,25] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 728.27it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 296.86it/s]


Partial density matrices computed in 3.64 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 64601.88it/s]


Gram matrix for training set computed in 0.75 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:02<00:00, 142.19it/s]

Gram matrix for testing set computed in 2.82 seconds.


In [3]:
L = 51
bond = 150
p = partial(L = L, bond = bond)

In [5]:
keep = [0,1,2,3,4,5] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 539.02it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 344.21it/s]


Partial density matrices computed in 3.52 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 65188.21it/s]


Gram matrix for training set computed in 0.74 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:02<00:00, 149.80it/s]

Gram matrix for testing set computed in 2.67 seconds.


In [6]:
keep = [24] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 372.71it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 363.23it/s]


Partial density matrices computed in 3.41 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 562502.43it/s]


Gram matrix for training set computed in 0.09 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 1452.63it/s]

Gram matrix for testing set computed in 0.28 seconds.


In [7]:
keep = [24,25,26,27,28] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 531.18it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:01<00:00, 328.74it/s]


Partial density matrices computed in 3.54 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 311585.70it/s]


Gram matrix for training set computed in 0.16 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:00<00:00, 707.10it/s]

Gram matrix for testing set computed in 0.57 seconds.


In [8]:
keep = [24,25,26,27,28,29] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 464.25it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:00<00:00, 470.45it/s]


Partial density matrices computed in 4.28 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 59266.49it/s]


Gram matrix for training set computed in 0.82 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:02<00:00, 140.07it/s]

Gram matrix for testing set computed in 2.86 seconds.
